In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import glob
import os

In [2]:
allFiles = sorted(glob.glob("*.txt"),key=os.path.getmtime) # includes all files into a single stucture,
# Sorted by modification time so that the first measurement is first...
print(allFiles)

['RealSHGData_THzDelay_1_07092017.txt', 'RealSHGData_THzDelay_2_07092017.txt', 'RealSHGData_THzDelay_3_07092017.txt', 'RealSHGData_THzDelay_4_07092017.txt', 'RealSHGData_THzDelay_5_07092017.txt', 'RealSHGData_THzDelay_6_07092017.txt', 'RealSHGData_THzDelay_7_07092017.txt', 'RealSHGData_THzDelay_8_07092017.txt', 'RealSHGData_THzDelay_9_07092017.txt', 'RealSHGData_THzDelay_10_07092017.txt']


In [3]:
frame = pd.DataFrame()
df = pd.DataFrame()

In [4]:
for i,file_ in enumerate(allFiles):
#    f = open(file_,'r')
#     for j,line in enumerate(f):
#         if j == 1:
#             date = line[14:22]
#             break
    df = pd.read_csv(file_,index_col = None, header=None, skiprows = 10, skipfooter = 1, delimiter = '\t')
#     frame[file_] = df[5]
    df['file']=file_
    df['timeDelay']= df[1]
    df['PDvalue'] = df[3]
#     df['date']=date
    if i < 1:
        frame = df
    else:
        frame = frame.append(df)

/Users/sanjaykhatri/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.


In [5]:
c = 3*10**11; #Speed of Light in mm/s, Going to use this to convert spatial information to time. 
T1 = (347*0.000125 + 0.0229)*2*25.4/(10**-12*c)
def ConvertToTimeDelay(x,itr,offset):
    c = 3*10**11
    timeDelay = []
    for i in range(0,len(x)):
        timedelay = -((x[i]*itr + offset)*25.4*2/(10**-12*c))+T1
        timeDelay.append(timedelay)
    return(timeDelay)

In [ ]:
times = pd.Series()
PDvalues = pd.Series()
AvgPD10 = pd.Series()
itr = 0.000125
offset = 0.023
for i in range(0,900):
    time = frame[1][i]
    PDvalue = frame[3][i]
    times = times.append(time)
    PDvalues = PDvalues.append(PDvalue)
#print(len(PDvalues))
    
times = times.values.reshape(900,len(allFiles))
PDvalues = PDvalues.values.reshape(900,len(allFiles)) 
timeDelay = ConvertToTimeDelay(times,itr,offset) # function converts to timedelay
PDvalues = PDvalues.astype(np.float) #Ensures all the values are floats so that I can plot them!
AvgPD10 = PDvalues.mean(axis = 1)

plt.plot(timeDelay,AvgPD10,'b.')
plt.xlabel('Stage Delay (ps)')
plt.ylabel('PD Signal (arb)')
plt.grid(True)
plt.show()
